In [19]:
import numpy as np
import pandas as pd
import sys
import os
import pickle 

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
import scipy.stats as ss

In [20]:
sys.path.append('../utils')
from simple_impute import simple_imputer

# Task Specifics

In [21]:
INTERVENTION = 'vent'
RANDOM = 0
MAX_LEN = 240
SLICE_SIZE = 6
GAP_TIME = 6
PREDICTION_WINDOW = 4
OUTCOME_TYPE = 'all'
NUM_CLASSES = 4

In [22]:
CHUNK_KEY = {'ONSET': 0, 'CONTROL': 1, 'ON_INTERVENTION': 2, 'WEAN': 3}

# Load Data

In [23]:
DATAFILE = '../data/curated/all_hourly_data.h5'

In [24]:
DATAFILE

'../data/curated/all_hourly_data.h5'

In [25]:
X = pd.read_hdf(DATAFILE,'vitals_labs')
Y = pd.read_hdf(DATAFILE,'interventions')
static = pd.read_hdf(DATAFILE,'patients')

In [26]:
Y = Y[[INTERVENTION]]

In [27]:
print ('Shape of X : ', X.shape)
print ('Shape of Y : ', Y.shape)
print ('Shape of static : ', static.shape)

Shape of X :  (2200954, 312)
Shape of Y :  (2200954, 1)
Shape of static :  (34472, 29)


# Preprocessing Data

## Train-Test Split, Stratified

In [28]:
train_ids, test_ids = train_test_split(static.reset_index(), test_size=0.2, 
                                       random_state=RANDOM, stratify=static['mort_hosp'])
split_train_ids, val_ids = train_test_split(train_ids, test_size=0.125, 
                                            random_state=RANDOM, stratify=train_ids['mort_hosp'])

## Imputation and Standardization of Time Series Features

In [18]:
X_clean = simple_imputer(X,train_ids['subject_id'])

/Users/wesleypickering/UIUC/healthcare/MIMIC_Extract/notebooks/../utils/simple_impute.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
/Users/wesleypickering/UIUC/healthcare/MIMIC_Extract/notebooks/../utils/simple_impute.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
/Users/wesleypickering/UIUC/healthcare/MIMIC_Extract/notebooks/../ut

In [29]:
def minmax(x):# normalize
    mins = x.min()
    maxes = x.max()
    x_std = (x - mins) / (maxes - mins)
    return x_std

In [30]:
def std_time_since_measurement(x):
    idx = pd.IndexSlice
    x = np.where(x==100, 0, x)
    means = x.mean()
    stds = x.std()
    x_std = (x - means)/stds
    return x_std

In [31]:
idx = pd.IndexSlice
X_std = X_clean.copy()
X_std.loc[:,idx[:,'mean']] = X_std.loc[:,idx[:,'mean']].apply(lambda x: minmax(x))
X_std.loc[:,idx[:,'time_since_measured']] = X_std.loc[:,idx[:,'time_since_measured']].apply(lambda x: std_time_since_measurement(x))

In [32]:
X_std.columns = X_std.columns.droplevel(-1)

In [33]:
del X

## Categorization of Static Features

In [34]:
def categorize_age(age):
    if age > 10 and age <= 30: 
        cat = 1
    elif age > 30 and age <= 50:
        cat = 2
    elif age > 50 and age <= 70:
        cat = 3
    else: 
        cat = 4
    return cat

def categorize_ethnicity(ethnicity):
    if 'AMERICAN INDIAN' in ethnicity:
        ethnicity = 'AMERICAN INDIAN'
    elif 'ASIAN' in ethnicity:
        ethnicity = 'ASIAN'
    elif 'WHITE' in ethnicity:
        ethnicity = 'WHITE'
    elif 'HISPANIC' in ethnicity:
        ethnicity = 'HISPANIC/LATINO'
    elif 'BLACK' in ethnicity:
        ethnicity = 'BLACK'
    else: 
        ethnicity = 'OTHER'
    return ethnicity

In [35]:
# use gender, first_careunit, age and ethnicity for prediction
static_to_keep = static[['gender', 'age', 'ethnicity', 'first_careunit', 'intime']]
static_to_keep.loc[:, 'intime'] = static_to_keep['intime'].astype('datetime64').apply(lambda x : x.hour)
static_to_keep.loc[:, 'age'] = static_to_keep['age'].apply(categorize_age)
static_to_keep.loc[:, 'ethnicity'] = static_to_keep['ethnicity'].apply(categorize_ethnicity)
static_to_keep = pd.get_dummies(static_to_keep, columns = ['gender', 'age', 'ethnicity', 'first_careunit'])

/var/folders/by/jd5hsptn06169y6mhx1d27hm0000gn/T/ipykernel_27951/2554866979.py:3: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  static_to_keep.loc[:, 'intime'] = static_to_keep['intime'].astype('datetime64').apply(lambda x : x.hour)
/var/folders/by/jd5hsptn06169y6mhx1d27hm0000gn/T/ipykernel_27951/2554866979.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_to_keep.loc[:, 'intime'] = static_to_keep['intime'].astype('datetime64').apply(lambda x : x.hour)
/var/folders/by/jd5hsptn06169y6mhx1d27hm0000gn/T/ipykernel_27951/2554866979.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

## Create Feature Matrix

In [36]:
# merge time series and static data
X_merge = pd.merge(X_std.reset_index(), static_to_keep.reset_index(), on=['subject_id','icustay_id','hadm_id'])
# add absolute time feature
abs_time = (X_merge['intime'] + X_merge['hours_in'])%24
X_merge.insert(4, 'absolute_time', abs_time)
X_merge.drop('intime', axis=1, inplace=True)
X_merge = X_merge.set_index(['subject_id','icustay_id','hadm_id','hours_in'])

In [37]:
del X_std, X_clean

## Make Tensors

In [38]:
def create_x_matrix(x):
    zeros = np.zeros((MAX_LEN, x.shape[1]-4))
    x = x.values
    x = x[:(MAX_LEN), 4:]
    zeros[0:x.shape[0], :] = x
    return zeros

def create_y_matrix(y):
    zeros = np.zeros((MAX_LEN, y.shape[1]-4))
    y = y.values
    y = y[:,4:]
    y = y[:MAX_LEN, :]
    zeros[:y.shape[0], :] = y
    return zeros

In [39]:
x = np.array(list(X_merge.reset_index().groupby('subject_id').apply(create_x_matrix)))
y = np.array(list(Y.reset_index().groupby('subject_id').apply(create_y_matrix)))[:,:,0]

In [40]:
lengths = np.array(list(X_merge.reset_index().groupby('subject_id').apply(lambda x: x.shape[0])))

In [41]:
keys = pd.Series(X_merge.reset_index()['subject_id'].unique())

In [42]:
print("X tensor shape: ", x.shape)
print("Y tensor shape: ", y.shape)
print("lengths shape: ", lengths.shape)

X tensor shape:  (34472, 240, 330)
Y tensor shape:  (34472, 240)
lengths shape:  (34472,)


## Stratified Sampling

In [43]:
train_indices = np.where(keys.isin(train_ids['subject_id']))[0]
test_indices = np.where(keys.isin(test_ids['subject_id']))[0]
train_static = train_ids
split_train_indices = np.where(keys.isin(split_train_ids['subject_id']))[0]
val_indices = np.where(keys.isin(val_ids['subject_id']))[0]

In [44]:
X_train = x[split_train_indices]
Y_train = y[split_train_indices]
X_test = x[test_indices]
Y_test = y[test_indices]
X_val = x[val_indices]
Y_val = y[val_indices]
lengths_train = lengths[split_train_indices]
lengths_val = lengths[val_indices]
lengths_test = lengths[test_indices]

In [45]:
print("Training size: ", X_train.shape[0])
print("Validation size: ", X_val.shape[0])
print("Test size: ", X_test.shape[0])

Training size:  24129
Validation size:  3448
Test size:  6895


## Make Windows

In [46]:
def make_3d_tensor_slices(X_tensor, Y_tensor, lengths):

    num_patients = X_tensor.shape[0]
    timesteps = X_tensor.shape[1]
    num_features = X_tensor.shape[2]
    X_tensor_new = np.zeros((lengths.sum(), SLICE_SIZE, num_features + 1))
    Y_tensor_new = np.zeros((lengths.sum()))

    current_row = 0
    
    for patient_index in range(num_patients):
        x_patient = X_tensor[patient_index]
        y_patient = Y_tensor[patient_index]
        length = lengths[patient_index]

        for timestep in range(length - PREDICTION_WINDOW - GAP_TIME - SLICE_SIZE):
            x_window = x_patient[timestep:timestep+SLICE_SIZE]
            y_window = y_patient[timestep:timestep+SLICE_SIZE]
            x_window = np.concatenate((x_window, np.expand_dims(y_window,1)), axis=1)
            result_window = y_patient[timestep+SLICE_SIZE+GAP_TIME:timestep+SLICE_SIZE+GAP_TIME+PREDICTION_WINDOW]
            result_window_diff = set(np.diff(result_window))
            #if 1 in result_window_diff: pdb.set_trace()
            gap_window = y_patient[timestep+SLICE_SIZE:timestep+SLICE_SIZE+GAP_TIME]
            gap_window_diff = set(np.diff(gap_window))

            #print result_window, result_window_diff

            if OUTCOME_TYPE == 'binary':
                if max(gap_window) == 1:
                    result = None
                elif max(result_window) == 1:
                    result = 1
                elif max(result_window) == 0:
                    result = 0
                if result != None:
                    X_tensor_new[current_row] = x_window
                    Y_tensor_new[current_row] = result
                    current_row += 1

            else: 
                if 1 in gap_window_diff or -1 in gap_window_diff:
                    result = None
                elif (len(result_window_diff) == 1) and (0 in result_window_diff) and (max(result_window) == 0):
                    result = CHUNK_KEY['CONTROL']
                elif (len(result_window_diff) == 1) and (0 in result_window_diff) and (max(result_window) == 1):
                    result = CHUNK_KEY['ON_INTERVENTION']
                elif 1 in result_window_diff: 
                    result = CHUNK_KEY['ONSET']
                elif -1 in result_window_diff:
                    result = CHUNK_KEY['WEAN']
                else:
                    result = None

                if result != None:
                    X_tensor_new[current_row] = x_window
                    Y_tensor_new[current_row] = result
                    current_row += 1

    X_tensor_new = X_tensor_new[:current_row,:,:]
    Y_tensor_new = Y_tensor_new[:current_row]

    return X_tensor_new, Y_tensor_new

In [47]:
x_train, y_train = make_3d_tensor_slices(X_train, Y_train, lengths_train)
x_val, y_val = make_3d_tensor_slices(X_val, Y_val, lengths_val)
x_test, y_test = make_3d_tensor_slices(X_test, Y_test, lengths_test)

In [48]:
y_train_classes = label_binarize(y_train, classes=range(NUM_CLASSES))
y_val_classes = label_binarize(y_val, classes=range(NUM_CLASSES))
y_test_classes = label_binarize(y_test, classes=range(NUM_CLASSES))

In [49]:
del X_train, Y_train, X_test, Y_test, X_val, Y_val

In [50]:
print('shape of x_train: ', x_train.shape)
print('shape of x_val: ', x_val.shape)
print('shape of x_test: ', x_test.shape)

shape of x_train:  (1107487, 6, 331)
shape of x_val:  (161031, 6, 331)
shape of x_test:  (314555, 6, 331)


# Random Forest and Logistic Regression

## Prepare data

In [33]:
static_col = 17 #static_to_keep.shape[1] - 1
time_series_col = 124 #X_merge.shape[1] - static_col _shape_repr(array.shape)

In [34]:
def remove_duplicate_static(x):
    x_static = x[:,0,time_series_col:x.shape[2]-1]
    x_timeseries = np.reshape(x[:,:,:time_series_col],(x.shape[0], -1))
    x_int = x[:,:,-1]
    x_concat = np.concatenate((x_static, x_timeseries, x_int), axis=1)
    return x_concat

In [35]:
# concatenate hourly features
x_train_concat = remove_duplicate_static(x_train)
x_val_concat = remove_duplicate_static(x_val)
x_test_concat = remove_duplicate_static(x_test)

In [36]:
print(x_train_concat.shape)
print(x_val_concat.shape)
print(x_test_concat.shape)

(1107493, 767)
(161025, 767)
(314548, 767)


## Hyperparameter Generation

In [48]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out
    
class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [49]:
N = 10
np.random.seed(RANDOM)
LR_dist = DictDist({
    'C': Choice(np.geomspace(1e-3, 1e3, 10000)),
    'penalty': Choice(['l2']),
    'solver': Choice(['sag']),
    'max_iter': Choice([100, 200]),
    'class_weight': Choice(['balanced']),
    'multi_class': Choice(['multinomial']),
    'random_state': Choice([RANDOM])
})
LR_hyperparams_list = LR_dist.rvs(N)
        
RF_dist = DictDist({
    'n_estimators': ss.randint(50, 200),
    'max_depth': ss.randint(2, 10),
    'min_samples_split': ss.randint(2, 75),
    'min_samples_leaf': ss.randint(1, 50),
    'class_weight': Choice(['balanced']),
    'random_state': Choice([RANDOM])
})
RF_hyperparams_list = RF_dist.rvs(N)

## Fit model

In [56]:
def run_basic(model, hyperparams_list, X_train, X_val, X_test):
    best_s, best_hyperparams = -np.Inf, None
    for i, hyperparams in enumerate(hyperparams_list):
        M = model(**hyperparams)
        M.fit(X_train, y_train)
        s = roc_auc_score(y_val_classes, M.predict_proba(X_val),average='macro')
        if s > best_s:
            best_s, best_hyperparams = s, hyperparams

    return run_only_final(model, best_hyperparams, X_train, X_val, X_test)

def run_only_final(model, best_hyperparams, X_train, X_val, X_test):
    best_M = model(**best_hyperparams)
    best_M.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)))
    y_pred  = best_M.predict_proba(X_test)
    idx = np.argmax(y_pred, axis=-1)
    y_pred_label = np.zeros(y_pred.shape)
    y_pred_label[np.arange(y_pred_label.shape[0]), idx] = 1
    auc   = roc_auc_score(y_test_classes, y_pred, average=None)
    aucmacro = roc_auc_score(y_test_classes, y_pred, average='macro')
    accuracy =  accuracy_score(y_test_classes, y_pred_label)
    f1 = f1_score(y_test_classes, y_pred_label, average='macro')
    auprc = average_precision_score(y_test_classes, y_pred_label, average='macro')
    return auc, aucmacro, accuracy, f1, auprc

In [57]:
results = {}
for model_name, model, hyperparams_list in [('RF', RandomForestClassifier, RF_hyperparams_list), 
                                            ('LR', LogisticRegression, LR_hyperparams_list)]:
    if model_name not in results: results[model_name] = {}

    results[model_name] = run_basic(
        model, hyperparams_list, x_train_concat, x_val_concat, x_test_concat)
    print("Final results for model %s, (AUC, Macro_AUC, Accuracy, F1 Macro, AUPRC Macro)" % (model_name))
    print(results[model_name])

Final results for model RF, (AUC, Macro_AUC, Accuracy, F1 Macro, AUPRC Macro)
(array([0.8707733 , 0.98981498, 0.9852656 , 0.93999635]), 0.9464625569824558, 0.7965493342828439, 0.4810231595666129, 0.42696393873684396)


/home/shirlyw/anaconda3/envs/mimic_extract_py36/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/shirlyw/anaconda3/envs/mimic_extract_py36/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/shirlyw/anaconda3/envs/mimic_extract_py36/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/shirlyw/anaconda3/envs/mimic_extract_py36/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/shirlyw/anacon

Final results for model LR, (AUC, Macro_AUC, Accuracy, F1 Macro, AUPRC Macro)
(array([0.71889636, 0.98294004, 0.98429292, 0.93236335]), 0.9046231696007481, 0.7849231277897173, 0.4772524408862778, 0.43060796540254614)


# CNN

In [53]:
import tensorflow as tf


import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Reshape, RepeatVector, Lambda
from keras.layers import Input, Conv2D, Conv1D, Conv3D, MaxPooling2D, MaxPooling1D
from keras.layers import Concatenate
from keras import backend as K
from keras.callbacks import EarlyStopping

import random as rn

In [54]:
BATCH_SIZE = 128
EPOCHS = 12
DROPOUT = 0.5

In [56]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
#class_weight = [1,1,1,1]
class_weight = dict(zip(range(len(class_weight)), class_weight))

In [67]:
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
K.set_session(sess)

np.random.seed(RANDOM)
tf.random.set_seed(RANDOM)
rn.seed(RANDOM)

input_shape = (x_train.shape[1], x_train.shape[2])
inputs = Input(shape=input_shape)
model = Conv1D(64, kernel_size=3,
                 strides=1,
                 activation='relu',
                 input_shape=input_shape,
                 padding='same',
                 name='conv2')(inputs)

model = (MaxPooling1D(pool_size=3, strides=1))(model)

model2 = Conv1D(64, kernel_size=4,
                 strides=1,
                 activation='relu',
                 input_shape=input_shape,
                 padding='same',
                 name='conv3')(inputs)

model2 = MaxPooling1D(pool_size=3, strides=1)(model2)

model3 = Conv1D(64, kernel_size=5,
                 strides=1,
                 activation='relu',
                 input_shape=input_shape,
                 padding='same',
                 name='conv4')(inputs)

model3 = MaxPooling1D(pool_size=3, strides=1)(model3)

models = [model, model2, model3]

full_model = keras.layers.concatenate(models)
full_model = Flatten()(full_model)
full_model = Dense(128, activation='relu')(full_model)
full_model = Dropout(DROPOUT)(full_model)
full_model = Dense(NUM_CLASSES, activation='softmax')(full_model)

full_model = keras.models.Model(inputs=inputs, outputs=full_model)

full_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=.0005),
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

full_model.fit(x_train, y_train_classes,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          class_weight=class_weight,
        #   callbacks=[early_stopping],
          validation_data=(x_val, y_val_classes))

/Users/wesleypickering/anaconda3/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/12
8653/8653 [==============================] - 90s 10ms/step - loss: 0.8686 - accuracy: 0.6174 - val_loss: 0.7258 - val_accuracy: 0.6225
Epoch 2/12
8653/8653 [==============================] - 86s 10ms/step - loss: 0.7889 - accuracy: 0.6390 - val_loss: 0.7382 - val_accuracy: 0.6255
Epoch 3/12
8653/8653 [==============================] - 87s 10ms/step - loss: 0.7622 - accuracy: 0.6487 - val_loss: 0.7268 - val_accuracy: 0.6100
Epoch 4/12
8653/8653 [==============================] - 87s 10ms/step - loss: 0.7442 - accuracy: 0.6529 - val_loss: 0.5846 - val_accuracy: 0.7845
Epoch 5/12
8653/8653 [==============================] - 87s 10ms/step - loss: 0.7269 - accuracy: 0.6622 - val_loss: 0.6354 - val_accuracy: 0.6924
Epoch 6/12
8653/8653 [==============================] - 87s 10ms/step - loss: 0.7130 - accuracy: 0.6629 - val_loss: 0.7010 - val_accuracy: 0.6321
Epoch 7/12
8653/8653 [==============================] - 87s 10ms/step - loss: 0.6987 - accuracy: 0.6659 - val_loss: 0.7623 -

In [68]:
test_preds_cnn = full_model.predict(x_test, batch_size=BATCH_SIZE)
idx = np.argmax(test_preds_cnn, axis=-1)
test_preds_cnn_label = np.zeros(test_preds_cnn.shape)
test_preds_cnn_label[np.arange(test_preds_cnn_label.shape[0]), idx] = 1
print("AUC:")
print(roc_auc_score(y_test_classes, test_preds_cnn, average=None))
print("AUC Macro:")
print(roc_auc_score(y_test_classes, test_preds_cnn, average='macro'))
print("Accuracy: ")
print(accuracy_score(y_test_classes, test_preds_cnn_label))
print("F1 Macro:")
print(f1_score(y_test_classes, test_preds_cnn_label, average='macro'))
print("AUPRC Macro: ")
print(average_precision_score(y_test_classes, test_preds_cnn_label, average='macro'))

2458/2458 [==============================] - 12s 5ms/step
AUC:
[0.68766231 0.9785553  0.98024821 0.92805958]
AUC Macro:
0.8936313499832
Accuracy: 
0.6881753588402664
F1 Macro:
0.461604334631575
AUPRC Macro: 
0.42827299103513994
